<a href="https://colab.research.google.com/github/son3012/2021watson/blob/main/gridsearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grid Search 실습

### 데이터셋 다운로드

In [7]:
! wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1GBtNoT3sC2ZjscrPKaFwb5pBEqLNrF9Z' -O pima-indians-diabetes.csv #https://drive.google.com/file/d/1GBtNoT3sC2ZjscrPKaFwb5pBEqLNrF9Z/view?usp=sharing

--2021-05-31 08:29:57--  https://docs.google.com/uc?export=download&id=1GBtNoT3sC2ZjscrPKaFwb5pBEqLNrF9Z
Resolving docs.google.com (docs.google.com)... 173.194.215.139, 173.194.215.138, 173.194.215.113, ...
Connecting to docs.google.com (docs.google.com)|173.194.215.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0o-1s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/72sum80urknmirt0gkgve5sldn4mtu6e/1622449725000/04591858293269658552/*/1GBtNoT3sC2ZjscrPKaFwb5pBEqLNrF9Z?e=download [following]
--2021-05-31 08:29:58--  https://doc-0o-1s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/72sum80urknmirt0gkgve5sldn4mtu6e/1622449725000/04591858293269658552/*/1GBtNoT3sC2ZjscrPKaFwb5pBEqLNrF9Z?e=download
Resolving doc-0o-1s-docs.googleusercontent.com (doc-0o-1s-docs.googleusercontent.com)... 74.125.26.132, 2607:f8b0:400c:c04::84
Connecting to doc-0o-1s-docs.googleusercontent.com (doc-0o-1

## batch size와 epoch 최적화

In [8]:
#@title
# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model():
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=8, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.710938 using {'batch_size': 20, 'epochs': 100}
0.584635 (0.024774) with: {'batch_size': 10, 'epochs': 10}
0.656250 (0.014616) with: {'batch_size': 10, 'epochs': 50}
0.687500 (0.016573) with: {'batch_size': 10, 'epochs': 100}
0.513021 (0.028587) with: {'batch_size': 20, 'epochs': 10}
0.656250 (0.035943) with: {'batch_size': 20, 'epochs': 50}
0.710938 (0.019918) with: {'batch_size': 20, 'epochs': 100}
0.587240 (0.064529) with: {'batch_size': 40, 'epochs': 10}
0.658854 (0.013279) with: {'batch_size': 40, 'epochs': 50}
0.670573 (0.008027) with: {'batch_size': 40, 'epochs': 100}
0.576823 (0.001841) with: {'batch_size': 60, 'epochs': 10}
0.598958 (0.007366) with: {'batch_size': 60, 'epochs': 50}
0.640625 (0.019401) with: {'batch_size': 60, 'epochs': 100}
0.635417 (0.030978) with: {'batch_size': 80, 'epochs': 10}
0.630208 (0.045814) with: {'batch_size': 80, 'epochs': 50}
0.660156 (0.006379) with: {'batch_size': 80, 'epochs': 100}
0.550781 (0.057675) with: {'batch_size': 100, 'epochs':

## 학습률과 모멘텀 최적화

In [9]:
#@title
# Use scikit-learn to grid search the learning rate and momentum
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
import time
# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.01, momentum=0):
	# create model
	modelx = Sequential()
	modelx.add(Dense(12, input_dim=8, activation='relu'))
	modelx.add(Dense(1, activation='sigmoid'))
	# Compile model
	optimizer = SGD(lr=learn_rate, momentum=momentum)
	modelx.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return modelx
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
modelx = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=modelx, param_grid=param_grid, n_jobs=-1, cv=3)

# 트레이닝 시작
start = time.time()
print("트레이닝 시작")
grid_result = grid.fit(X, Y)
end = time.time()
print(end - start, ' 초 결렸습니다')    

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

트레이닝 시작


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


453.6333496570587  초 결렸습니다
Best: 0.690104 using {'learn_rate': 0.001, 'momentum': 0.4}
0.675781 (0.027251) with: {'learn_rate': 0.001, 'momentum': 0.0}
0.673177 (0.042353) with: {'learn_rate': 0.001, 'momentum': 0.2}
0.690104 (0.024150) with: {'learn_rate': 0.001, 'momentum': 0.4}
0.679688 (0.013902) with: {'learn_rate': 0.001, 'momentum': 0.6}
0.667969 (0.020915) with: {'learn_rate': 0.001, 'momentum': 0.8}
0.660156 (0.038273) with: {'learn_rate': 0.001, 'momentum': 0.9}
0.674479 (0.025976) with: {'learn_rate': 0.01, 'momentum': 0.0}
0.673177 (0.035849) with: {'learn_rate': 0.01, 'momentum': 0.2}
0.648438 (0.025315) with: {'learn_rate': 0.01, 'momentum': 0.4}
0.649740 (0.026557) with: {'learn_rate': 0.01, 'momentum': 0.6}
0.649740 (0.026557) with: {'learn_rate': 0.01, 'momentum': 0.8}
0.651042 (0.024774) with: {'learn_rate': 0.01, 'momentum': 0.9}
0.651042 (0.024774) with: {'learn_rate': 0.1, 'momentum': 0.0}
0.651042 (0.024774) with: {'learn_rate': 0.1, 'momentum': 0.2}
0.651042 (0.0

In [10]:
# Use scikit-learn to grid search the learning rate and momentum
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import SGD
import time
# Function to create model, required for KerasClassifier
def create_model(learn_rate=0.01, momentum=0):
	# create model
	modelx = Sequential()
	modelx.add(Dense(12, input_dim=8, activation='relu'))
	modelx.add(Dense(1, activation='sigmoid'))
	# Compile model
	optimizer = SGD(lr=learn_rate, momentum=momentum)
	modelx.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return modelx
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
modelx = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
# learn_rate와 momentum 두 개의 하이퍼파라미터의 그리드를 코드로 정의하십시오. 
# learn_rate는 0.001, 0.01. 0.1, 0.2, 0.3로 하고 momentum은 0.0, 0.2, 0.4, 0.6, 0.8, 0.9로 하십시오.
learn_rate = (0.001)
momentum = (0.2)
param_grid =dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=modelx, param_grid=param_grid, n_jobs=-1, cv=3)
# 트레이닝 시작
start = time.time()
print("트레이닝 시작")
grid_result = grid.fit(X, Y)
end = time.time()
print(end - start, ' 초 결렸습니다') 

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

트레이닝 시작


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


81.73087167739868  초 결렸습니다
Best: 0.678385 using {'batch_size': 80, 'epochs': 50}
0.654948 (0.029635) with: {'batch_size': 10, 'epochs': 10}
0.657552 (0.031948) with: {'batch_size': 10, 'epochs': 50}
0.648438 (0.019401) with: {'batch_size': 10, 'epochs': 100}
0.669271 (0.018136) with: {'batch_size': 20, 'epochs': 10}
0.674479 (0.011201) with: {'batch_size': 20, 'epochs': 50}
0.671875 (0.019401) with: {'batch_size': 20, 'epochs': 100}
0.654948 (0.008027) with: {'batch_size': 40, 'epochs': 10}
0.651042 (0.047877) with: {'batch_size': 40, 'epochs': 50}
0.657552 (0.017566) with: {'batch_size': 40, 'epochs': 100}
0.648438 (0.019918) with: {'batch_size': 60, 'epochs': 10}
0.636719 (0.044194) with: {'batch_size': 60, 'epochs': 50}
0.653646 (0.034104) with: {'batch_size': 60, 'epochs': 100}
0.635417 (0.018688) with: {'batch_size': 80, 'epochs': 10}
0.678385 (0.011201) with: {'batch_size': 80, 'epochs': 50}
0.665365 (0.023510) with: {'batch_size': 80, 'epochs': 100}
0.639323 (0.016053) with: {'b

In [11]:
layer = tf.keras.layers.Dropout(rate=0.2, input_shape=(2,))

NameError: ignored

In [ ]:
layer = tf.keras.layers.LeakyReLU(alpha=0.1)